In [1]:
from tdc.single_pred import Tox
import deepchem
import xgboost
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.EState import EState

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [3]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
ecfpFeat = deepchem.deepchem.feat.CircularFingerprint(radius=4)
convMolFeat = deepchem.deepchem.feat.ConvMolFeaturizer()
xgb_reg_ecfp = xgboost.XGBRegressor()
xgb_reg_estate = xgboost.XGBRegressor()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('Half_Life_Obach') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    #trainMol = train.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    #validMol = valid.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    train_valMol = train_val.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost
    #trainFeat = np.stack(np.array(trainMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    #validFeat = np.stack(np.array(validMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    testFeat = np.stack(np.array(testMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    train_valFeat = np.stack(np.array(train_valMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))

    #featurize training, valid, and test data for xgboost
    #ecfp_f_train = ecfpFeat.featurize(train.iloc[:,1].to_list())
    #ecfp_f_valid = ecfpFeat.featurize(valid.iloc[:,1].to_list())
    ecfp_f_test = ecfpFeat.featurize(test.iloc[:,1].to_list())
    train_val_f = ecfpFeat.featurize(train_val.iloc[:,1].to_list())

    #featurize training, valid, and test data for the GCN
    cv_f_train = convMolFeat.featurize(train.iloc[:,1].to_list())
    cv_f_valid = convMolFeat.featurize(valid.iloc[:,1].to_list())
    cv_f_test = convMolFeat.featurize(test.iloc[:,1].to_list())

    #convert training and validation data into a deepchem dataset for the gcn
    gcn_train_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    gcn_valid_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    #fit data on GCN
    reg = deepchem.deepchem.models.GraphConvModel(
        n_tasks=1, 
        dropout=.0005,
        dense_layer_size=1063,
        graph_conv_layers=[128, 128, 128],
        mode="regression",)
    callback = deepchem.deepchem.models.ValidationCallback(gcn_valid_data, 1000, metric)
    reg.fit(gcn_train_data, nb_epoch=100, callbacks=callback)

    #predict values on gcn and reshape array (reshape value is 20% of dataset size)
    gcn_pred = reg.predict_on_batch(X=np.array(cv_f_test)).reshape(135,)

    #fit xgboost model and store np ndarray in xgb_pred
    xgb_reg_estate.fit(X=train_valFeat, y=train_val.iloc[:,2], eval_metric="mae", verbose=True)
    xgb_reg_ecfp.fit(X=train_val_f, y=train_val.iloc[:,2], eval_metric="mae")
    pred_estate = xgb_reg_estate.predict(X=testFeat)
    pred_ecfp = xgb_reg_ecfp.predict(X=ecfp_f_test)

    # store test predictions in y_pred_test
    y_pred_test = np.mean([ pred_estate, pred_ecfp, gcn_pred ], axis=0)

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 532/532 [00:00<00:00, 892.64it/s]
generating training, validation splits...


predictions for run #
1
[  7.3329163  16.689756   -5.28814    12.743428  228.93037  ]


100%|██████████| 532/532 [00:00<00:00, 685.48it/s]


generating training, validation splits...


predictions for run #
2
[  9.605119   11.839496   -7.6092834  24.340628  232.90898  ]


...

predictions for run #
3
[ -1.2999271   1.4079098  -6.702572    7.570189  222.81213  ]


100%|██████████| 532/532 [00:01<00:00, 525.50it/s]
generating training, validation splits...


predictions for run #
4
[-10.677241   -4.0239005 -18.894411   -0.7932625 215.6707   ]


100%|██████████| 532/532 [00:00<00:00, 685.01it/s]


predictions for run #
5
[  0.21465428   9.909673    -9.550147    10.688465   210.71965   ]
Prediction List:
[{'half_life_obach': array([ 7.33291626e+00,  1.66897564e+01, -5.28813982e+00,  1.27434282e+01,
        2.28930374e+02,  2.03179764e+02,  3.63544350e+01,  3.79046593e+01,
        6.49870529e+01,  1.57844725e+01,  3.49250374e+01, -4.08050919e+00,
        1.42298813e+01,  2.35669460e+01,  5.37364340e+00,  6.07545662e+00,
        2.57618011e+02,  2.86859035e+01, -3.04081478e+01, -2.93726711e+01,
       -2.93726711e+01,  1.75942097e+01,  1.13793592e+01,  5.12447815e+01,
        1.69944210e+01, -1.17324007e+00,  4.30517845e+01, -1.89641571e+00,
        1.09769220e+01,  2.21279587e+02,  1.20285006e+01,  2.77515545e+01,
       -2.12186184e-02, -1.31223230e+01,  2.75041580e+00,  3.03202701e+00,
        1.69972019e+01,  1.50884476e+01,  2.01992435e+01,  1.15436859e+01,
        5.84188890e+00,  6.82082951e-01,  2.02811265e+00, -1.24263115e+01,
        2.26360264e+01,  8.28561401e+00,  6.91

In [4]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'half_life_obach': {'spearman': 0.251}}
{'half_life_obach': {'spearman': 0.28}}
{'half_life_obach': {'spearman': 0.259}}
{'half_life_obach': {'spearman': 0.273}}
{'half_life_obach': {'spearman': 0.328}}


{'half_life_obach': [0.278, 0.027]}